In [1]:
# 📦 Import required libraries
import pyodbc # for commecting to sql
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from datetime import date
from matplotlib import dates as dates
import datetime as dt
import pyodbc
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import rcParams
import pandas as pd
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve
import implicit

In [3]:
# 🔌 Connect to the database and query data
conn = pyodbc.connect('Driver={SQL Server};'
'Server=DESKTOP-MB976LQ;'
'Database=AdventureWorksDW2017;'
'Trusted_Connection=yes;')
sql_for_df = "select CustomerKey, ProductKey, SalesOrderNumber,SalesAmount as Price,OrderQuantity from dbo.FactInternetSales"
sale = pd.read_sql(sql_for_df , conn)
conn.close()

C:\Users\anuma\AppData\Local\Temp\ipykernel_2652\1842265064.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sale = pd.read_sql(sql_for_df , conn)


In [4]:
# ➕ Code block
sale.head()

,CustomerKey,ProductKey,SalesOrderNumber,Price,OrderQuantity
0,21768,310,SO43697,3578.2700,1
1,28389,346,SO43698,3399.9900,1
2,25863,346,SO43699,3399.9900,1
3,14501,336,SO43700,699.0982,1
4,11003,346,SO43701,3399.9900,1


In [5]:
# 🔌 Connect to the database and query data
conn = pyodbc.connect('Driver={SQL Server};'
'Server=DESKTOP-MB976LQ;'
'Database=AdventureWorksDW2017;'
'Trusted_Connection=yes;')
sql_for_df = "select ProductKey, EnglishProductName as ProductName from dbo.DimProduct"
prod = pd.read_sql(sql_for_df , conn)
conn.close()

C:\Users\anuma\AppData\Local\Temp\ipykernel_2652\2370933051.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  prod = pd.read_sql(sql_for_df , conn)


In [7]:
# ➕ Code block
prod.head()

,ProductKey,ProductName
0,1,Adjustable Race
1,2,Bearing Ball
2,3,BB Ball Bearing
3,4,Headset Ball Bearings
4,5,Blade


In [8]:
# 🔗 Merge dataframes to enrich with product names
meg = prod.merge(sale, on="ProductKey")
meg.head()

,ProductKey,ProductName,CustomerKey,SalesOrderNumber,Price,OrderQuantity
0,214,"Sport-100 Helmet, Red",16313,SO51180,34.99,1
1,214,"Sport-100 Helmet, Red",12132,SO51181,34.99,1
2,214,"Sport-100 Helmet, Red",12390,SO51191,34.99,1
3,214,"Sport-100 Helmet, Red",11241,SO51192,34.99,1
4,214,"Sport-100 Helmet, Red",11338,SO51207,34.99,1


In [9]:
# ➕ Code block
meg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60398 entries, 0 to 60397
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ProductKey        60398 non-null  int64  
 1   ProductName       60398 non-null  object 
 2   CustomerKey       60398 non-null  int64  
 3   SalesOrderNumber  60398 non-null  object 
 4   Price             60398 non-null  float64
 5   OrderQuantity     60398 non-null  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 3.2+ MB


In [10]:
# ➕ Code block
meg.shape

(60398, 6)

In [11]:
# ➕ Code block
#lookup table that keeps track of each item ID along with a description of that item
lookUp = meg[['ProductKey', 'ProductName']].drop_duplicates() # Only get unique item/description pairs

In [10]:
# ➕ Code block
lookUp.head()

,ProductKey,ProductName
0,214,"Sport-100 Helmet, Red"
2230,217,"Sport-100 Helmet, Black"
4315,222,"Sport-100 Helmet, Blue"
6440,225,AWC Logo Cap
8630,228,"Long-Sleeve Logo Jersey, S"


In [11]:
# ➕ Code block
lookUp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 0 to 60012
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ProductKey   158 non-null    int64 
 1   ProductName  158 non-null    object
dtypes: int64(1), object(1)
memory usage: 3.7+ KB


In [12]:
# ➕ Code block
meg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60398 entries, 0 to 60397
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ProductKey        60398 non-null  int64  
 1   ProductName       60398 non-null  object 
 2   CustomerKey       60398 non-null  int64  
 3   SalesOrderNumber  60398 non-null  object 
 4   Price             60398 non-null  float64
 5   OrderQuantity     60398 non-null  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 3.2+ MB


In [13]:
# ➕ Code block
meg = meg[['ProductKey', 'OrderQuantity', 'CustomerKey']] # Get rid of unnecessary info
grp = meg.groupby(['CustomerKey', 'ProductKey']).sum().reset_index() # Group together
grp.OrderQuantity.loc[grp.OrderQuantity == 0] = 1 # Replace a sum of zero purchases with a one to indicate purchased
gp = grp.query('OrderQuantity > 0') # Only get customers where purchase totals were positive

In [14]:
# ➕ Code block
meg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60398 entries, 0 to 60397
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   ProductKey     60398 non-null  int64
 1   OrderQuantity  60398 non-null  int64
 2   CustomerKey    60398 non-null  int64
dtypes: int64(3)
memory usage: 1.8 MB


In [15]:
# ➕ Code block
grp.dtypes

CustomerKey      int64
ProductKey       int64
OrderQuantity    int64
dtype: object

In [16]:
# ➕ Code block
gp.head()

,CustomerKey,ProductKey,OrderQuantity
0,11000,214,1
1,11000,344,1
2,11000,353,1
3,11000,485,1
4,11000,488,1


In [17]:
# 🧮 Create a sparse matrix for recommender system
#creating a sparse ratings matrix of users and items

customers = list(np.sort(gp.CustomerKey.unique())) # Get our unique customers
products = list(gp.ProductKey.unique()) # Get our unique products that were purchased
quantity = list(gp.OrderQuantity) # All of our purchases

rows = gp.CustomerKey.astype('category').cat.codes 

# Get the associated row indices
cols = gp.ProductKey.astype('category').cat.codes 

# Get the associated column indices
purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products)))

In [18]:
# ➕ Code block
#Let’s check our final matrix object
purchases_sparse

<18484x158 sparse matrix of type '<class 'numpy.intc'>'
	with 59051 stored elements in Compressed Sparse Row format>

In [52]:
# ➕ Code block
#this shows we have 18,484 customers with 158 items with a purchase of 59,051!
# from other analysis i have done, I know this is correct as IHG does have 18,484 customers

In [20]:
# 🧮 Create a sparse matrix for recommender system
matrix_size = purchases_sparse.shape[0]*purchases_sparse.shape[1] # Number of possible interactions in the matrix
num_purchases = len(purchases_sparse.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity

97.97803231806365

In [21]:
# 📦 Import required libraries
import random

def make_train(ratings, pct_test = 0.2):

    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    random.seed(20) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered  

In [22]:
# 🧮 Create a sparse matrix for recommender system
product_train, product_test, product_users_altered = make_train(purchases_sparse, pct_test = 0.2)

In [23]:
# 🎓 Split data into training and test sets
def implicit_weighted_ALS(training_set, lambda_val = 0.1, alpha = 40, iterations = 10, rank_size = 20, seed = 0):

    
    # first set up our confidence matrix
    
    conf = (alpha*training_set) # To allow the matrix to stay sparse, I will add one later when each row is taken 
                                # and converted to dense. 
    num_user = conf.shape[0]
    num_item = conf.shape[1] # Get the size of our original ratings matrix, m x n
    
    # initialize our X/Y feature vectors randomly with a set seed
    rstate = np.random.RandomState(seed)
    
    X = sparse.csr_matrix(rstate.normal(size = (num_user, rank_size))) # Random numbers in a m x rank shape
    Y = sparse.csr_matrix(rstate.normal(size = (num_item, rank_size))) # Normally this would be rank x n but we can 
                                                                 # transpose at the end. Makes calculation more simple.
    X_eye = sparse.eye(num_user)
    Y_eye = sparse.eye(num_item)
    lambda_eye = lambda_val * sparse.eye(rank_size) # Our regularization term lambda*I. 
    
    # We can compute this before iteration starts.  
    # Begin iterations
   
    for iter_step in range(iterations): # Iterate back and forth between solving X given fixed Y and vice versa
        # Compute yTy and xTx at beginning of each iteration to save computing time
        yTy = Y.T.dot(Y)
        xTx = X.T.dot(X)
        # Being iteration to solve for X based on fixed Y
        for u in range(num_user):
            conf_samp = conf[u,:].toarray() # Grab user row from confidence matrix and convert to dense
            pref = conf_samp.copy() 
            pref[pref != 0] = 1 # Create binarized preference vector 
            CuI = sparse.diags(conf_samp, [0]) # Get Cu - I term, don't need to subtract 1 since we never added it 
            yTCuIY = Y.T.dot(CuI).dot(Y) # This is the yT(Cu-I)Y term 
            yTCupu = Y.T.dot(CuI + Y_eye).dot(pref.T) # This is the yTCuPu term, where we add the eye back in
                                                      # Cu - I + I = Cu
            X[u] = spsolve(yTy + yTCuIY + lambda_eye, yTCupu) 
            # Solve for Xu = ((yTy + yT(Cu-I)Y + lambda*I)^-1)yTCuPu, equation 4 from the paper  
        # Begin iteration to solve for Y based on fixed X 
        for i in range(num_item):
            conf_samp = conf[:,i].T.toarray() # transpose to get it in row format and convert to dense
            pref = conf_samp.copy()
            pref[pref != 0] = 1 # Create binarized preference vector
            CiI = sparse.diags(conf_samp, [0]) # Get Ci - I term, don't need to subtract 1 since we never added it
            xTCiIX = X.T.dot(CiI).dot(X) # This is the xT(Cu-I)X term
            xTCiPi = X.T.dot(CiI + X_eye).dot(pref.T) # This is the xTCiPi term
            Y[i] = spsolve(xTx + xTCiIX + lambda_eye, xTCiPi)
            # Solve for Yi = ((xTx + xT(Cu-I)X) + lambda*I)^-1)xTCiPi, equation 5 from the paper
    # End iterations
        return X, Y.T

# Transpose at the end to make up for not being transposed at the beginning. 
# Y needs to be rank x n. Keep these as separate matrices for scale reasons. 

In [24]:
# 🎓 Split data into training and test sets
user_vecs, item_vecs = implicit_weighted_ALS(product_train, lambda_val = 0.1, alpha = 15, iterations = 1,
                                            rank_size = 20)

In [25]:
# ➕ Code block
user_vecs[0,:].dot(item_vecs).toarray()[0,:5]

array([ 0.02578946, -0.01339369, -0.01591212, -0.00499155, -0.00279749])

In [26]:
# 📦 Import required libraries
import implicit

In [27]:
# ➕ Code block
alpha = 40
user_vecs, item_vecs = implicit.alternating_least_squares((product_train*alpha).astype('double'), 
                                                          factors=20, 
                                                          regularization = 0.1, 
                                                         iterations = 50)

This method is deprecated. Please use the AlternatingLeastSquares class instead


  0%|          | 0/50 [00:00<?, ?it/s]

### Evaluating the Recommender System

In [53]:
# 🎓 Split data into training and test sets
#training set had 20% of the purchases masked. This will allow us to evaluate the 
#performance of our recommender 
#system. Essentially, we need to see if the order of recommendations 
#given for each user matches the items they ended up purchasing. 
#A commonly used metric for this kind of problem is the area under the 
#Receiver Operating Characteristic (or ROC) curve. 
#A greater area under the curve means we are recommending items that 
#end up being purchased near the top of the list of 
#recommended items.

In [51]:
# 📦 Import required libraries
from sklearn import metrics

def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    parameters:
    - predictions: your prediction output
    - test: the actual target result you are comparing to
    returns:
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)   

In [30]:
# 📈 Evaluate model using AUC metric
#Now, utilize this helper function inside of a second function that will calculate the AUC for each user 
#in our training set that had at least one item masked. 
#It should also calculate AUC for the most popular items for our users to compare.

def calc_mean_auc(training_set, altered_users, predictions, test_set):

    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc)) 


# Return the mean AUC rounded to three decimal places for both test and popularity benchmark
#We can now use this function to see how our recommender system is doing. 
#To use this function, we will need to transform our output from the ALS function to csr_matrix 
#format and transpose the item vectors. The original pure Python version output the user and item vectors into the correct format already.

In [31]:
# 🎓 Split data into training and test sets
calc_mean_auc(product_train, product_users_altered, 
              [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
# AUC for our recommender system

(0.767, 0.831)

In [63]:
# 📦 Import required libraries
import numpy as np
from sklearn.metrics import mean_squared_log_error

def rmse(y_true, y_pred):
    np.sqrt(mean_squared_log_error(y_true, y_pred))

In [64]:
# ➕ Code block
rmse

<function __main__.rmse(y_true, y_pred)>

#### Trying out the code

In [32]:
# ➕ Code block
customers_arr = np.array(customers) # Array of customer IDs from the ratings matrix
products_arr = np.array(products) # Array of product IDs from the ratings matrix

In [33]:
# ➕ Code block
#Now, we can create a function that will return a list of the 
#item descriptions from our earlier created item lookup table

def get_items_purchased(CustomerKey, mf_train, customers_list, products_list, lookUp):
    
    cust_ind = np.where(customers_list == CustomerKey)[0][0]    # Returns the index row of our customer id
    purchased_ind = mf_train[cust_ind,:].nonzero()[1]        # Get column indices of purchased items
    prod_codes = products_list[purchased_ind]              # Get the stock codes for our purchased items
    return lookUp.loc[lookUp.ProductKey.isin(prod_codes)]

In [34]:
# ➕ Code block
#We need to look these up by a customer’s ID. Looking at the list of customers:
customers_arr[:4]

array([11000, 11001, 11002, 11003], dtype=int64)

In [35]:
# 🎓 Split data into training and test sets
get_items_purchased(11000, product_train, customers_arr, products_arr, lookUp)

,ProductKey,ProductName
0,214,"Sport-100 Helmet, Red"
14901,358,"Mountain-200 Black, 38"
16097,362,"Mountain-200 Black, 46"
19225,386,"Road-550-W Yellow, 42"
38118,486,All-Purpose Bike Stand


In [36]:
# 📦 Import required libraries
from sklearn.preprocessing import MinMaxScaler

In [37]:
# 🎓 Split data into training and test sets
def rec_items(customer_id, mf_train, user_vecs, item_vecs, customer_list, item_list, item_lookup, num_items = 10):
    
    cust_ind = np.where(customer_list == customer_id)[0][0] # Returns the index row of our customer id
    pref_vec = mf_train[cust_ind,:].toarray() # Get the ratings from the training set ratings matrix
    pref_vec = pref_vec.reshape(-1) + 1 # Add 1 to everything, so that items not purchased yet become equal to 1
    pref_vec[pref_vec > 1] = 0 # Make everything already purchased zero
    rec_vector = user_vecs[cust_ind,:].dot(item_vecs.T) # Get dot product of user vector and all item vectors
    # Scale this recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0] 
    recommend_vector = pref_vec*rec_vector_scaled 
    # Items already purchased have their recommendation multiplied by zero
    product_idx = np.argsort(recommend_vector)[::-1][:num_items] # Sort the indices of the items into order 
    # of best recommendations
    rec_list = [] # start empty list to store items
    for index in product_idx:
        code = item_list[index]
        rec_list.append([code, lookUp.ProductName.loc[lookUp.ProductKey == code].iloc[0]]) 
        # Append our descriptions to the list
    codes = [item[0] for item in rec_list]
    descriptions = [item[1] for item in rec_list]
    final_frame = pd.DataFrame({'StockCode': codes, 'Description': descriptions}) # Create a dataframe 
    return final_frame[['StockCode', 'Description']] # Switch order of columns around

In [38]:
# 🎓 Split data into training and test sets
rec_items(11000, product_train, user_vecs, item_vecs, customers_arr, products_arr, lookUp,
                       num_items = 3)

,StockCode,Description
0,580,"Road-350-W Yellow, 40"
1,583,"Road-350-W Yellow, 48"
2,538,LL Road Tire


In [39]:
# 🎓 Split data into training and test sets
get_items_purchased(11000, product_train, customers_arr, products_arr, lookUp)

,ProductKey,ProductName
0,214,"Sport-100 Helmet, Red"
14901,358,"Mountain-200 Black, 38"
16097,362,"Mountain-200 Black, 46"
19225,386,"Road-550-W Yellow, 42"
38118,486,All-Purpose Bike Stand


In [40]:
# 🎓 Split data into training and test sets
rec_items(11000, product_train, user_vecs, item_vecs, customers_arr, products_arr, lookUp,
                       num_items = 3)

,StockCode,Description
0,580,"Road-350-W Yellow, 40"
1,583,"Road-350-W Yellow, 48"
2,538,LL Road Tire


In [41]:
# 🎓 Split data into training and test sets
get_items_purchased(12410, product_train, customers_arr, products_arr, lookUp)

,ProductKey,ProductName
11622,314,"Road-150 Red, 56"
54837,560,"Touring-2000 Blue, 60"


In [42]:
# 🎓 Split data into training and test sets
rec_items(12410, product_train, user_vecs, item_vecs, customers_arr, products_arr, lookUp,
                       num_items = 3)

,StockCode,Description
0,599,"Mountain-500 Black, 48"
1,586,"Touring-3000 Blue, 50"
2,356,"Mountain-200 Silver, 46"


In [43]:
# 🎓 Split data into training and test sets
get_items_purchased(18479, product_train, customers_arr, products_arr, lookUp)

,ProductKey,ProductName
0,214,"Sport-100 Helmet, Red"
16097,362,"Mountain-200 Black, 46"
55881,571,"Touring-3000 Yellow, 58"


In [44]:
# 🎓 Split data into training and test sets
rec_items(18479, product_train, user_vecs, item_vecs, customers_arr, products_arr, lookUp,
                       num_items = 3)

,StockCode,Description
0,484,Bike Wash - Dissolver
1,560,"Touring-2000 Blue, 60"
2,487,Hydration Pack - 70 oz.


In [ ]:
# ➕ Code block
